In [1]:
# This program performs Heurstic Optimization

In [2]:
# Loading the relevant libraries

import pandas as pd
import datetime
import math
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import SuitabilityCode as SC
import processing_utilities as pu
import warnings
warnings.filterwarnings('ignore')

In [3]:
import azureml.core
from azureml.core import Workspace, Dataset, Environment, Datastore
from azureml.data.datapath import DataPath

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.47.0 to work with eunmldevamlwsgom


In [4]:
# Get the datastores available
dataset = Dataset.get_by_name(ws, name='Auger_Backlog')
rawdf = dataset.to_pandas_dataframe()

In [5]:
# Configuration Settings
# Merging with suitability code or just use raw file
is_suitability_code_integration = 'True'
assetname = 'Auger'
totalweeks = 50

In [6]:
# rawfile = r'C:\Users\S.Chandrasekaran5\OneDrive - Shell\Documents\Work\Preventive Maintenace Optimization\VEGA\MILP2\GOM\Sprint 8\Ursa Backlog 11-22-22.xlsx'
# suitabilitycodefile = r'C:\Users\S.Chandrasekaran5\OneDrive - Shell\Documents\Work\Preventive Maintenace Optimization\VEGA\MILP2\GOM\Sprint 8\Ursa Sutibality Codes.xlsx'
# process_step_file = r'C:\Users\S.Chandrasekaran5\OneDrive - Shell\Documents\Work\Preventive Maintenace Optimization\VEGA\MILP2\GOM\Sprint 8\BAT Tool Source.xlsx'
# Process step File needed to map the code
process_step_file = 'BAT Tool Source.csv'

rawdf['UpdatedWorkCenter'] = rawdf['WORK_CENTER_DESCRIPTION']
rawdf.rename(columns = {'CONCAT_USER_STATUS':'Order User Status'}, inplace = True)
rawdf.rename(columns = {'WORK_ORDER_TYPE_CODE':'Order Type'}, inplace = True)
rawdf.rename(columns = {'PLANNED_WORK':'Work'}, inplace = True)
rawdf.rename(columns = {'BASIC_START_DATE':'Earliest start date'}, inplace = True)
rawdf.rename(columns = {'PRIORITY_CODE':'Priority'}, inplace = True)
rawdf.rename(columns = {'LATEST_ALLOWABLE_FINISH_DATE':'Latest Allowed Finish Date'}, inplace = True)
rawdf.rename(columns = {'WORK_ORDER_SRC_ID':'Order'}, inplace = True)

if is_suitability_code_integration =='True':
    # Merging Suitability and updating work centers
    rawdf = SC.integrate_with_suitabilitycode(rawdf,assetname)

In [7]:
# Adding the PS Levels
rawdf = SC.integrate_with_process_steps(rawdf,process_step_file)



PS LEVELS


EXEC    6728
PS-1    2113
PS-2    1706
PS-4    1181
PS-5     544
PS-3     109
Name: PSLevel, dtype: int64
Nulls :10


In [8]:
# Weekday Thursday = 3
# Weekday Wednesday = 2
# For URSA, Weekday is thursday
import math
def datetime_to_previous_weekstart(original_datetime):
    th=3
    if pd.isnull(original_datetime):
        return 
    if original_datetime.weekday() >=th:
        return original_datetime + datetime.timedelta(days=th-original_datetime.weekday())
    else:
        return original_datetime + datetime.timedelta(days=-th-1-original_datetime.weekday())
    


In [9]:
# Loading the capacity for URSA for the critical work crafts
if assetname=='URSA':
    capacityfile_withpath = 'ursa_capacity.csv'
if assetname=='Olympus':
    capacityfile_withpath = 'Olympus_capacity.csv'
if assetname=='Auger':
    capacityfile_withpath = 'Auger_capacity.csv'
# capacity = pu.load_capacity_info(capacityfile_withpath)
# capacity
# capacitydf = pd.DataFrame(index = range(0,totalweeks),columns=arr)
# for cnt in arr:
#     capacitydf[cnt] = capacity[cnt]
capacitydf = pd.read_csv(capacityfile_withpath)
capacitydf.set_index(['Week'], inplace=True)

In [10]:
rawdf

,ID,Order,FACILITY_ID,PLANNER_GROUP_CODE,MAINTENANCE_PLANNER_GROUP_NAME,ASSET_DOMAIN_CODE,WBS_ID,WBS_CODE,WORK_ORDER_NAME,Earliest start date,...,SuitabilityCode,UpdatedWorkCenter,OT,Status,EXEC,WP,ZPM,SFIX,Code,PSLevel
0,81897272,000029228140,3236971.0,426,Auger,AD0285,825622,00091910,CAIR2745 Hot Work Replace Sewage Overboa,2025-12-25,...,NaN,Construction (MainWorkCenter),C,R,F,O,F,A,CRFOFA,PS-5
1,88944962,000060117152,3236971.0,426,Auger,AD0285,825622,00091910,Replace MBV 430 Block Valves,2023-05-04,...,35.0,"Benoit, Donnie",C,R,F,O,Z,A,CRFOZA,PS-4
2,92726722,000060495948,3236971.0,426,Auger,AD0285,825622,00091910,CAIR6217 - Defective Handrail(s) UD SE 2,2023-09-25,...,NaN,Construction (MainWorkCenter),C,R,F,W,F,A,CRFWFA,PS-2
3,96705832,000060894488,3236971.0,426,Auger,AD0285,825622,00091910,Repair Grounding Cables on PRT's,2025-12-25,...,NaN,Auger GIS Instrumentation,C,R,F,O,F,A,CRFOFA,PS-5
4,97427852,000060966776,3236971.0,426,Auger,AD0285,825622,00091910,PBE-271 Engine Corrosion Repair,2023-05-16,...,NaN,EXT SVS-For PM CNF-Call Out,C,R,F,O,Z,A,CRFOZA,PS-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12386,4649612652,000062683447,3236971.0,426,Auger,AD0285,825622,00091910,2W Blowout Preventer Pressure Test,2023-06-14,...,NaN,Auger H & P Drilling,P,R,E,F,F,A,PREFFA,EXEC
12387,4649615712,000062683753,3236971.0,426,Auger,AD0285,825622,00091910,WK Single Fall Lifeboat Inspection,2023-06-15,...,15.0,Electrician,P,R,E,F,F,A,PREFFA,EXEC
12388,4649616992,000062683881,3236971.0,426,Auger,AD0285,825622,00091910,2Y Yokogawa Battery Replacement,2023-04-07,...,5.0,ET/CAO,P,R,E,F,F,A,PREFFA,EXEC
12389,4649617382,000062683920,3236971.0,426,Auger,AD0285,826682,00092016,1100082698/Riser Cleaning,2024-10-02,...,NaN,Maintenance & Integrity Marine,P,C,F,W,F,A,PCFWFA,PS-2


In [11]:
# PreProcessing
# URSA
from datetime import timedelta

# Interchaning Priorities

rawdf['Priority'].replace('S','-1',inplace=True)
rawdf['Priority'].replace('E','0',inplace=True)

# Assigning Week Numbers to ESD
    
rawdf['Earliest start date'] = pd.to_datetime(rawdf['Earliest start date'],errors='coerce')
rawdf['nearestE'] = rawdf.apply(lambda row: datetime_to_previous_weekstart(row['Earliest start date']), axis=1)
rawdf['Total_Days'] = ( rawdf['nearestE'] - datetime_to_previous_weekstart(datetime.datetime.today()) ).dt.days +1
rawdf['WeekNumberE'] = (rawdf['Total_Days']/7) 
rawdf['ESD_WeekNumber'] = rawdf['WeekNumberE'].apply(np.int)


# # Assigning Week Numbers to LAFD

rawdf['Latest Allowed Finish Date'] = pd.to_datetime(rawdf['Latest Allowed Finish Date'],errors='coerce')
rawdf['nearestL'] = rawdf.apply(lambda row: datetime_to_previous_weekstart(row['Latest Allowed Finish Date']), axis=1)
rawdf['Total_Days'] = ( rawdf['nearestL'] - datetime_to_previous_weekstart(datetime.datetime.today()) ).dt.days +1
rawdf['WeekNumberL'] = (rawdf['Total_Days']/7) 
interindex = rawdf[~rawdf['WeekNumberL'].isnull()].index
rawdf.loc[interindex,'LAFD_WeekNumber'] = rawdf.loc[interindex,'WeekNumberL'].apply(math.ceil)

# Dropping intermediate columns so that outputs looks fine
rawdf = rawdf.drop(['nearestE','nearestL','WeekNumberE','WeekNumberL','Total_Days'],axis=1)

# Defining the scope of the problem 
# only 72FP
ind = rawdf[ (rawdf['Order Type'] == '72FP') | (rawdf['Order Type'] == '72FC') | (rawdf['Order Type'] == 'GEN') | (rawdf['Order Type'] == 'ZADM')| (rawdf['Order Type'] == '71PO')].index
rawdf.loc[set(rawdf.index) - set(ind),'Allocated'] = 999



In [12]:
rawdf[rawdf['Allocated'] != 999]['Order Type'].value_counts()

72FP    5658
72FC    3486
71PO    2195
ZADM     494
GEN      366
Name: Order Type, dtype: int64

In [13]:
# Concentrating only the important WOs

if assetname == 'URSA':
    arr = ['ACR/Instrument Technician','Electrician','Mechanic','ET/CAO','Ursa Utilities Crane Mechanic','Ursa Utilities Solar Mechanic','Ursa Bilfinger Maintenance Crew']

if assetname == 'Olympus':
    arr = ['ACR/Instrument Technician','Electrician','Mechanic','ET/CAO','Crane Mechanic','Turbine Mechanic','Olympus Bilfinger Maintenance Crew']

if assetname == 'Auger':
    arr = ['ACR/Instrument Technician','Electrician','Mechanic','ET/CAO','Crane Mechanic','Turbine Mechanic','Auger Bilfinger Maintenance Crew']


df = rawdf[rawdf['Allocated'] != 999]
df = df.dropna(subset=['Work','Earliest start date'],axis=0)


# Scoping the constrainer
df = df[df['ESD_WeekNumber'] >=0 ]
df = df[df['ESD_WeekNumber'] <=50 ]
df = df[df['UpdatedWorkCenter'].isin(arr)]
rawdf.loc[set(rawdf.index) - set(df.index),'Allocated'] = 999


In [14]:
# Initial allocation

df['Allocated'] = 1
df['AllocatedReason'] = ''
df['MoveReason'] = ''
df['Capreqd'] = ''
df['UnAllocatedReason'] = ''
df['PriorityFilling'] = ''
df['AlternateESD'] = df['ESD_WeekNumber']


# Initialize CapcityTrackingdataframe
capacitytrackingdf = capacitydf.copy()

In [15]:
# WORK ORDER OF INTEREST
# psdf = SC.weekwise_pslevels(df[df['Order Type'] == '72FC'],'ESD_WeekNumber')
# SC.plotting_bar_graph(psdf,'Original')
# plt.savefig('orginal.png')
# psdf = SC.weekwise_pslevels(df[df['Order Type'] == '72FP'],'ESD_WeekNumber')
# SC.plotting_bar_graph(psdf)
# print(len(df))

In [16]:
# To convert the unloading point to datetime
# from dateutil.parser import parse

# def is_date(string, fuzzy=True):
#     """
#     Return whether the string can be interpreted as a date.

#     :param string: str, string to check for date
#     :param fuzzy: bool, ignore unknown tokens in string if True
#     """
#     try: 
#         parse(string, fuzzy=fuzzy)
#         return True

#     except ValueError:
#         return False
    
# unloadingdf = df[(df['RESERVATION_UNLOADING_POINT'].str.contains('ETA')) | (df['RESERVATION_UNLOADING_POINT'].str.contains('eta'))]    
# for cnt in range(0,len(unloadingdf)):
#     strtocheck = str(unloadingdf.loc[unloadingdf.index[cnt],'RESERVATION_UNLOADING_POINT'])
#     if is_date(strtocheck) == True :
#         df.loc[unloadingdf.index[cnt],'ETADate'] = parse(strtocheck,fuzzy=True)
        
# Converting ETA to Weeknumber
        
df['ETADate'] = pd.to_datetime(df['ETADate'],errors='coerce')
df['nearestE'] = df.apply(lambda row: datetime_to_previous_weekstart(row['ETADate']), axis=1)
df['Total_Days'] = ( df['nearestE'] - datetime_to_previous_weekstart(datetime.datetime.today()) ).dt.days +1
df['WeekNumberE'] = (df['Total_Days']/7) 
interindex = df[~df['WeekNumberE'].isnull()].index
df.loc[interindex,'ETA_WeekNumber'] = df.loc[interindex,'WeekNumberE'].apply(np.int)


# Dropping intermediate columns so that outputs looks fine
df = df.drop(['nearestE','WeekNumberE','Total_Days'],axis=1)

## ZPBL Constraint - CAM Constraint - T0 Constraint

In [17]:
workorder_zpbl = df[( (df['ESD_WeekNumber'] == 0) | (df['Order User Status'].str.contains('ZPBL')) | (df['Order User Status'].str.contains('CAM'))) & (df['Order Type'] == '72FC') ]['Order'].unique()
# Stagnant df
stagnantdf = pd.DataFrame()
for cnt in workorder_zpbl:
    stagnantdf = pd.concat([stagnantdf,df[df['Order'] == cnt]], axis=0)
stagnantdf['Allocated'] = 9
stagnantdf['AllocatedReason'] = 'ZPBL/CAM/T0 Constraint'

# Reduction of Capacity by zpbl
for i in workorder_zpbl:
    for cnt in range(0,50):
        for count in arr:
            val = df[ ((df['ESD_WeekNumber']==cnt) & (df['UpdatedWorkCenter']==count) & (df['Order Type'] == '72FC') & (df['Order'] == i))]['Work'].sum()
            capacitydf.loc[cnt,count] = capacitydf.loc[cnt,count] - val

# Removing them from the df
df=df[~df.index.isin(stagnantdf.index)]


In [18]:
# Update CapcityTrackingdataframe
capacitytrackingdf = pu.update_capacity_tracking_dataframe(capacitytrackingdf, capacitydf,'_1')

In [19]:
# Dealing with WorkOrders
#****************** PS COnstraints *******************

listofallT4psdfwhole = df[ ((df['PSLevel'] == 'PS-1') |  (df['PSLevel'] == 'PS-2') |  (df['PSLevel'] == 'PS-3')) & (df['Order Type'] == '72FC') & (df['ESD_WeekNumber'] <= 3)]

listofallT4psdfresidual = listofallT4psdfwhole[listofallT4psdfwhole['ESD_WeekNumber'] == listofallT4psdfwhole['LAFD_WeekNumber'] ]

listofallT4psdf = listofallT4psdfwhole[listofallT4psdfwhole['ESD_WeekNumber'] < listofallT4psdfwhole['LAFD_WeekNumber'] ]
listofallT4psdf['AlternateESD'] = listofallT4psdf['ESD_WeekNumber'] + np.ceil(0.8*(listofallT4psdf['LAFD_WeekNumber'] - listofallT4psdf['ESD_WeekNumber']))
listofallT4psdf.loc[listofallT4psdf.index,'MoveReason'] = 'Changed due to PS 1-3'
listofallT4psdf.loc[listofallT4psdf.index,'Allocated'] = 2

# Reduce the capacity in the moved week.
for i in listofallT4psdf['Order'].unique():
    for cnt in range(0,50):
        for count in arr:
            val = listofallT4psdf[ (listofallT4psdf['AlternateESD']==cnt) & (listofallT4psdf['UpdatedWorkCenter']==count) & (listofallT4psdf['Order Type'] == '72FC') & listofallT4psdf['Order'] == i]['Work'].sum()
            capacitydf.loc[cnt,count] = capacitydf.loc[cnt,count] - val

# Removing them from the df
df=df[~df.index.isin(listofallT4psdf.index)]



# Moving them to alternate week - fixed case
# processleveldf.loc[processleveldf[ processleveldf['ESD_WeekNumber'] <= 3].index,'AlternateESD'] = 5
#****************** PS COnstraints *******************

listofallT4psdfresidual.loc[listofallT4psdfresidual.index,'Allocated'] = 29
listofallT4psdfresidual.loc[listofallT4psdfresidual.index,'PriorityFilling'] = ''
# Removing them from the df
df=df[~df.index.isin(listofallT4psdfresidual.index)]

In [20]:
#****************** ETA COnstraints - 2 *******************

etadf1 = df[ (df['PSLevel'] == 'PS-4') & (( df['ESD_WeekNumber'] - df['ETA_WeekNumber'] ) < 3) & (df['ETA_WeekNumber'] + 3 <= df['LAFD_WeekNumber'])]

etadf1['AlternateESD'] = etadf1['ETA_WeekNumber'] + 3
etadf1['Allocated'] = 3
etadf1['MoveReason'] = 'ETA Constraint'
etadf1['PriorityFilling'] = ''

# Reduce the capacity in the moved week.
for i in etadf1['Order'].unique():
    for cnt in range(0,50):
        for count in arr:
            val = etadf1[ (etadf1['AlternateESD']==cnt) & (etadf1['UpdatedWorkCenter']==count) & (etadf1['Order Type'] == '72FC') & etadf1['Order'] == i]['Work'].sum()
            capacitydf.loc[cnt,count] = capacitydf.loc[cnt,count] - val

# Removing them from the df
df=df[~df.index.isin(etadf1.index)]

# Residual 
etadf1Residual = df[ (df['PSLevel'] == 'PS-4') & (( df['ESD_WeekNumber'] - df['ETA_WeekNumber'] ) <3) ]
etadf1Residual['AlternateESD'] = etadf1Residual['ESD_WeekNumber'] 
etadf1Residual['Allocated'] = 39
etadf1Residual['PriorityFilling'] = ''
# Removing them from the df
df=df[~df.index.isin(etadf1Residual.index)]



In [21]:
# Update CapcityTrackingdataframe
capacitytrackingdf = pu.update_capacity_tracking_dataframe(capacitytrackingdf, capacitydf,'_2')

In [22]:
# Reduction of Capacity by PMs, PO , GEN, ZADM
for cnt in range(0,50):
    for count in arr:
        val = df[ (df['ESD_WeekNumber']==cnt) & (df['UpdatedWorkCenter']==count) & ( (df['Order Type'] == '72FP') | (df['Order Type'] == 'GEN') | (df['Order Type'] == 'ZADM')| (df['Order Type'] == '71PO'))]['Work'].sum()
        capacitydf.loc[cnt,count] = capacitydf.loc[cnt,count] - val


In [23]:
# Update CapcityTrackingdataframe
capacitytrackingdf = pu.update_capacity_tracking_dataframe(capacitytrackingdf, capacitydf,'_3')

In [24]:
# Capacity Requirement for the CMs
CMcapacityreqdf = pd.DataFrame()
for cnt in range(0,50):
    for count in arr:
        val = df[ (df['ESD_WeekNumber']==cnt) & (df['UpdatedWorkCenter']==count) &  (df['Order Type'] == '72FC') ]['Work'].sum()
        CMcapacityreqdf.loc[cnt,count] = val


In [25]:
# Update CapcityTrackingdataframe
capacitytrackingdf = pu.update_capacity_tracking_dataframe(capacitytrackingdf, CMcapacityreqdf,'_4')

In [26]:
# Seggregating CM versus the rest stagnant order types

FCdf = df[df['Order Type'] == '72FC']
FPdf = df[ ((df['Order Type'] == '72FP') | (df['Order Type'] == 'GEN') | (df['Order Type'] == 'ZADM')| (df['Order Type'] == '71PO'))]
FPdf['Allocated']=0


In [27]:
def perform_allocation(tempdf,FCdf,weeknumber,capacitydf):
    
    FCdf.loc[tempdf.index,'Allocated'] = 0
    FCdf.loc[tempdf.index,'AlternateESD'] = weeknumber
    # Reduce the capacity
    for gg in tempdf['UpdatedWorkCenter'].unique():
        
        FCdf.loc[tempdf[tempdf['UpdatedWorkCenter'] == gg].index,'AllocatedReason'] = 'Available Capacity for this WorkCenter in the Week :' + str(weeknumber) + ': '+ str(capacitydf.loc[weeknumber,gg]) +',Required Capacity for this WorkCenter in the Week :' +str(weeknumber) + ' : '+ str(tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum()) + '. LAFD Constraint Satisified. Bundling of WO Satsified'
        capacitydf.loc[weeknumber,gg] = capacitydf.loc[weeknumber,gg] - tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum()
         
        
    return FCdf,capacitydf
            
def iscapacityavailableforWO(capacitydf,tempdf,weeknumber):

    doable = 0
    for gg in tempdf['UpdatedWorkCenter'].unique():
#         print(weeknumber,gg,tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum(),capacitydf.loc[weeknumber,gg])
        if tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum() <= capacitydf.loc[weeknumber,gg] :
            doable = doable + 1           
        else:
            return False
    if doable > 0 :
        return True
    else:
        return False
        
def reasonfornotfilling(tempdf,FCdf,weeknumber,capacitydf):
    for gg in tempdf['UpdatedWorkCenter'].unique():
        FCdf.loc[tempdf[tempdf['UpdatedWorkCenter'] == gg].index,'MoveReason'] = FCdf.loc[tempdf[tempdf['UpdatedWorkCenter'] == gg].index,'MoveReason'] + 'Available Capacity in Week ' +str(weeknumber) +': ' +str(capacitydf.loc[weeknumber,gg]) +',Required Capacity in Week '+str(weeknumber) + ':' + str(tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum())
        FCdf.loc[tempdf[tempdf['UpdatedWorkCenter'] == gg].index,'Capreqd'] = str(tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum())
        
    return FCdf

In [28]:
def updatevalidation_matrix_before(validation_matrix,tempdf,capdf,weekno,linenumber):


    for gg in tempdf['UpdatedWorkCenter'].unique():
        validation_matrix.loc[linenumber,'Week'] = weekno
        validation_matrix.loc[linenumber,'WO'] = tempdf['Order'].unique()[0]
        validation_matrix.loc[linenumber,'WorkCraft'] = gg
        validation_matrix.loc[linenumber,'RequiredCapacity'] = tempdf[tempdf['UpdatedWorkCenter'] == gg]['Work'].sum()
        validation_matrix.loc[linenumber,'AvailableCapacity'] = capdf.loc[weekno,gg]
        validation_matrix.loc[linenumber,'PriorityWO'] =  tempdf.loc[tempdf.index[0],'Priority']
        validation_matrix.loc[linenumber,'ESD_WeekNumber'] = tempdf.loc[tempdf.index[0],'ESD_WeekNumber']
        validation_matrix.loc[linenumber,'LAFD_WeekNumber'] = tempdf.loc[tempdf.index[0],'LAFD_WeekNumber']
        linenumber=linenumber+1
        
    
    return validation_matrix,linenumber

def updatevalidation_matrix_after(valdf,tempdf,capdf,weekno,linenumber,priorityfilling):
    
    for gg in tempdf['UpdatedWorkCenter'].unique():
        valdf.loc[linenumber,'AvailableCapacity_afterallocation'] = capdf.loc[weekno,gg]
        valdf.loc[linenumber,'priorityfilling'] = priorityfilling   
        linenumber=linenumber+1
        


    return valdf
          
    

In [29]:
validation_matrix = pd.DataFrame(columns=['Week','WO','WorkCraft','RequiredCapacity','AvailableCapacity','AvailableCapacity_afterallocation','priorityfilling','ESD_WeekNumber','LAFD_WeekNumber'])
uplineno=0

In [30]:
# FOr Vignesh Math Optimization
Vigneshdf = FCdf[['Order','PLANNER_GROUP_CODE','MAINTENANCE_PLANNER_GROUP_NAME','Earliest start date','Order Type','Latest Allowed Finish Date','UpdatedWorkCenter','Priority','Work','WORK_ACTIVITY_NUMBER']]
Vigneshdf.rename(columns = {'UpdatedWorkCenter':'MAIN_WORK_CENTER_DESCRIPTION'},inplace=True)
Vigneshdf.to_csv('MathOptimization_Input_7_March_2023.csv')

In [31]:
for cnt in range(0,50):
#     print(cnt)
    k=1
    refdf = FCdf[ (FCdf['ESD_WeekNumber'] <= cnt) & (FCdf['LAFD_WeekNumber'] >= cnt)]
    refdf = refdf[refdf['Allocated']==1]
    refdf = refdf[ (refdf['ESD_WeekNumber']) <= (refdf['LAFD_WeekNumber'])]
    refdf= refdf.sort_values(by = ['Priority','LAFD_WeekNumber','Work'], ascending=[True,True,True])

    for count in refdf['Order'].unique():    
        iscapacityavailable = iscapacityavailableforWO(capacitydf,refdf[refdf['Order']==count],cnt) 
        
        lineno = uplineno
        validation_matrix,uplineno = updatevalidation_matrix_before(validation_matrix,refdf[refdf['Order']==count],capacitydf,cnt,uplineno)
        
        if  iscapacityavailable == True:
            curr_wo = refdf[refdf['Order']==count]
            FCdf,capacitydf = perform_allocation(curr_wo,FCdf,cnt,capacitydf)
            FCdf.loc[curr_wo.index,'PriorityFilling'] = k
            
            validation_matrix = updatevalidation_matrix_after(validation_matrix,refdf[refdf['Order']==count],capacitydf,cnt,lineno,k)
            
            k=k+1
        else:
            curr_wo = refdf[refdf['Order']==count]
            FCdf =  reasonfornotfilling(curr_wo,FCdf,cnt,capacitydf)



In [32]:
# Reason for Unallocated Status
unallocdf = FCdf[FCdf['Allocated']==1]

# FCdf.loc[FCdf['ESD_WeekNumber'].isnull().index,'UnAllocatedReason'] = ' LAFD lesser than ESD. This item is Unscheduled'
# FCdf.loc[FCdf['LAFD_WeekNumber'].isnull().index,'UnAllocatedReason'] = ' LAFD lesser than ESD. This item is Unscheduled'

FCdf.loc[unallocdf.index,'UnAllocatedReason'] = ' WO Bundling with the available capacity not Possible within LAFD. This item is Unscheduled'
ind = unallocdf[unallocdf['ESD_WeekNumber'] > unallocdf['LAFD_WeekNumber']].index
FCdf.loc[ind,'UnAllocatedReason'] = ' LAFD lesser than ESD. This item is Unscheduled'
# ind = unallocdf[unallocdf['MoveReason'] != ''].index


In [33]:
# Going back to df 

df.loc[FCdf.index,:] = FCdf.loc[FCdf.index,:]
df = pd.concat([df,listofallT4psdf], axis=0)
# df = pd.concat([df,etadf], axis=0)
df = pd.concat([df,etadf1], axis=0)
# df = pd.concat([df,etadf3], axis=0)
df = pd.concat([df,etadf1Residual], axis=0)
# df = pd.concat([df,etadf3Residual], axis=0)
df = pd.concat([df,listofallT4psdfresidual], axis=0)

df = pd.concat([df,stagnantdf], axis=0)
df.loc[FPdf.index,:] = FPdf.loc[FPdf.index,:]

df.loc[:,'MoveReason']  = ' '

inde = df[ ((df['ESD_WeekNumber'] != df['AlternateESD']) & (df['Allocated'] == 0) & (df['AlternateESD'] <= df['LAFD_WeekNumber']) ) ].index
df.loc[inde,'MovedOut'] = 1
df.loc[inde,'MoveReason']  = 'Moved from the scheduled week to meet the capacity for the WO' 

inde = df[ (df['Allocated'] == 2) ].index
df.loc[inde,'MovedOut'] = 1
df.loc[inde,'MoveReason']  = 'PS 1-3 Constraint. Any PS 1-3 WOs moved to 0.8 times LAFD'

inde = df[ (df['Allocated'] == 3) ].index
df.loc[inde,'MovedOut'] = 1
df.loc[inde,'MoveReason']  = 'ETA Constraint '

inde = df[ (df['Allocated'] == 39) ].index
df.loc[inde,'MoveReason']  = 'ETA-Unmovable'

inde = df[ (df['Allocated'] == 29) ].index
df.loc[inde,'MoveReason']  = 'PS1-3 Unmovable'

In [34]:
# Capacity Requirement for the CMs after optimization
CMcapacityreqdf_after = pd.DataFrame()
for cnt in range(0,50):
    for count in arr:
        val = df[ (df['AlternateESD']==cnt) & (df['UpdatedWorkCenter']==count)& (df['Allocated']==0) &  (df['Order Type'] == '72FC') ]['Work'].sum()
        CMcapacityreqdf_after.loc[cnt,count] = val

# Update CapcityTrackingdataframe
capacitytrackingdf = pu.update_capacity_tracking_dataframe(capacitytrackingdf, CMcapacityreqdf_after,'_5')

capacitytrackingdf.to_csv('CapacityTracking.csv')

In [35]:
addncols = list(set(df.columns)-set(rawdf.columns))
for cnt in addncols:
    rawdf[cnt]=''
rawdf.loc[df.index,:] = df.loc[df.index,:]

In [36]:
# # rawdf.loc[:,'MAIN_WORK_CENTER_DESCRIPTION'] = rawdf.loc[:,'UpdatedWorkCenter']
# rawdf.rename(column = {'Order User Status':'CONCAT_USER_STATUS'}, inplace = True)
# rawdf.rename(columns = {'Order Type':'WORK_ORDER_TYPE_CODE'}, inplace = True)
# rawdf.rename(columns = {'Work':'PLANNED_WORK'}, inplace = True)
# rawdf.rename(columns = {'Earliest start date':'BASIC_START_DATE'}, inplace = True)
# rawdf.rename(columns = {'Priority':'PRIORITY_CODE'}, inplace = True)
# rawdf.rename(columns = {'Latest Allowed Finish Date':'LATEST_ALLOWABLE_FINISH_DATE'}, inplace = True)
# rawdf.rename(columns = {'Order':'WORK_ORDER_SRC_ID'},

In [37]:
# Explanation of Allocated Codes
rawdf['Constraint'] = ''
rawdf.loc[rawdf[rawdf['Allocated']==0].index,'Constraint'] = 'No Change - Schedulable'
rawdf.loc[rawdf[ (rawdf['Allocated']==0) & (rawdf['MovedOut']==1) ].index,'Constraint'] = 'Change - Capacity Constraint overcome by Moving WO'
inn =  rawdf[(rawdf['Allocated']==1) & (rawdf['UnAllocatedReason'].str.contains('ESD') )].index
rawdf.loc[inn,'Constraint'] = 'No Change - LAFD / ESD Issue'
inn =  rawdf[(rawdf['Allocated']==1) & (rawdf['UnAllocatedReason'].str.contains('Bundling') )].index
rawdf.loc[inn,'Constraint'] = 'No Change - Capacity Constraint due to LAFD'
rawdf.loc[rawdf[rawdf['Allocated']==39].index,'Constraint'] = 'No Change - LAFD / ESD Issue'
rawdf.loc[rawdf[rawdf['Allocated']==29].index,'Constraint'] = 'No Change - LAFD / ESD Issue'
rawdf.loc[rawdf[rawdf['Allocated']==2].index,'Constraint'] = 'Change - Process Level Constraint'
rawdf.loc[rawdf[rawdf['Allocated']==3].index,'Constraint'] = 'Change - ETA Constraint'
rawdf.loc[rawdf[rawdf['Allocated']==9].index,'Constraint'] = 'No Change - ZPBL/CAM/T0 Constraint'
rawdf.loc[rawdf[rawdf['Allocated']==999].index,'Constraint'] = 'No Change - Out of Scope'

In [38]:
# Update the Validation_Matrix
# WOs which are Schedulable
schwo = rawdf[rawdf['Constraint'].str.contains('Schedulable')]['Order'].unique()
for wono in schwo:
    ind = validation_matrix[validation_matrix['WO']==wono].index
    validation_matrix.loc[ind,'Constraint'] = 'No Change - Schedulable'

schmvwo = rawdf[rawdf['Constraint'].str.contains('Moving WO')]['Order'].unique()
for wono in schmvwo:
    ind = validation_matrix[validation_matrix['WO']==wono].index
    validation_matrix.loc[ind,'Constraint'] = 'Change - Capacity Constraint overcome by Moving WO'
    
schimwo = rawdf[rawdf['Constraint'].str.contains('Capacity Constraint - Immovable')]['Order'].unique()
for wono in schimwo:
    ind = validation_matrix[validation_matrix['WO']==wono].index
    validation_matrix.loc[ind,'Constraint'] = 'No Change - Capacity Constraint due to LAFD'


In [39]:
rawdf.to_csv('AfterOptimization.csv')
validation_matrix.to_csv('validation_matrix.csv')

In [40]:
# # Data Egestion to the current optimization blob
defdatastore = ws.get_default_datastore()
# Register the dataset
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=rawdf, 
        name='AfterOptimization', 
        description='AfterOptimization',
        target=defdatastore
    )

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/ce257e5f-153d-4c17-b2cb-c616b47cad8b/


ExecutionError: 
Error Code: ScriptExecution.ReadDataFrame.StreamAccess.Validation
Validation Error Code: Invalid
Validation Target: PreppyFile
Failed Step: 0aecf2d9-0271-491a-9103-a01355f44305
Error Message: ScriptExecutionException was caused by ReadDataFrameException.
  Failed to read Pandas DataFrame form Python host. Make sure Dataflow is created directly from the source Pandas DataFrame.
    StreamAccessException was caused by ValidationException.
      Trying to read an invalid file. Missing sentinel value in the beginning
| session_id=dad8e622-9405-4350-a05b-303770cbe19f

In [ ]:
datastore = Datastore.get(ws, 'azblobsdk')
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=rawdf, 
        name='AfterOptimization', 
        description='AfterOptimization',
        target=datastore
    )